In [1]:
import os
import pandas as pd
import gspread
from datetime import datetime, timedelta
from threading import Thread
import time
ts = time.time()


In [2]:
def formatar_data(data) -> str:
    DATA_INICIAL = datetime(1899, 12, 30) 
    diferenca_datas = data - DATA_INICIAL
    data_formatada = diferenca_datas.days
    return data_formatada

def formatar_hora(hora) -> str:
    params = {
    'hours': hora.hour,
    'minutes': hora.minute,
    'seconds': hora.second}
    
    hora_formatada = timedelta(**params) / timedelta(days=1)
    return hora_formatada

def limpar_intervalo(aba,intervalo):
    
    aba.batch_clear([intervalo])
    
fname = os.path.abspath("./test-auto-330418-221ec3d586fa.json")
gc = gspread.service_account(fname)
sh = gc.open("Cópia de Acompanhamento de pedidos SETEMBRO 2021")

In [3]:
x = pd.read_excel('order_h.XLSX')

In [4]:
_bopisloja =  x[['Centro', 'Pedido Internet - Bemol On-line', 'Data do Pedido Bemol On-line']]

In [5]:
## Bopis Loja
def bopis_loja():
    bopisloja = _bopisloja.query('Centro > 0').copy()
    bopisloja['Data do Pedido Bemol On-line'] = bopisloja['Data do Pedido Bemol On-line'].apply(formatar_data)

    ## Lendo a aba 'BOPIS Recebido Loja'
    sh_bopisloja = sh.worksheet('BOPIS Recebido Loja')

    ## Atualiza a aba
    bopisloja_num = bopisloja.shape[0]+4
    limpar_intervalo(sh_bopisloja,'A4:C1500')
    sh_bopisloja.update(f'A4:C{bopisloja_num}',bopisloja.values.tolist())
    sh_bopisloja.format(f'C2:C{bopisloja_num}',
                        {'numberFormat':
                         {'type':
                          "DATE"
                          }
                        })

In [6]:
## Bopis Hora
def bopis_hora():
    bopishora = x[['Pedido Internet - Bemol On-line','Data do Pedido Bemol On-line',
                   'Hora do Pedido Bemol On-line','Caractere 1','Códido de Identificação do site']].copy()
    bopishora['Data do Pedido Bemol On-line'] = bopishora['Data do Pedido Bemol On-line'].apply(formatar_data)
    bopishora['Hora do Pedido Bemol On-line'] = bopishora['Hora do Pedido Bemol On-line'].apply(formatar_hora)
    bopishora.fillna('', inplace = True)

    ## Lendo a aba 'BOPIS por Hora'
    sh_bopishora = sh.worksheet('BOPIS por Hora')

    ## Atualiza a aba
    bopishora_num = bopishora.shape[0]+2
    limpar_intervalo(sh_bopishora,'A2:E15600')

    sh_bopishora.update(f'A2:E{bopishora_num}',bopishora.values.tolist())
    sh_bopishora.format(f'C2:C{bopishora_num}',
                        {'numberFormat':
                         {'type':
                          "TIME"
                          }
                        })
    sh_bopishora.format(f'B2:B{bopishora_num}',
                        {'numberFormat':
                         {'type':
                          "DATE"
                          }
                        })

In [7]:
## Base Pedidos
def base_pedidos():
    basepedidos = x[['Pedido Internet - Bemol On-line','Data do Pedido Bemol On-line',
                   'Hora do Pedido Bemol On-line','Códido de Identificação do site']].copy()

    basepedidos['Data do Pedido Bemol On-line'] = basepedidos['Data do Pedido Bemol On-line'].apply(formatar_data)
    basepedidos['Hora do Pedido Bemol On-line'] = basepedidos['Hora do Pedido Bemol On-line'].apply(formatar_hora)

    ## Lendo a aba 'Base PEDIDOS h/h'
    sh_basepedidos =  sh.worksheet('Base PEDIDOS h/h')

    ## Atualiza a aba
    basepedidos_num = basepedidos.shape[0]+2
    limpar_intervalo(sh_basepedidos,'A2:D2500')
    sh_basepedidos.update(f'A2:D{basepedidos_num}',basepedidos.values.tolist())
    sh_basepedidos.format(f'C2:C{basepedidos_num}',
                        {'numberFormat':
                         {'type':
                          "TIME"
                          }
                        })
    sh_basepedidos.format(f'B2:B{basepedidos_num}',
                        {'numberFormat':
                         {'type':
                          "DATE"
                          }
                        })

In [8]:
t1 = Thread(target=bopis_loja)
t2 = Thread(target=bopis_hora)
t3 = Thread(target=base_pedidos)

t1.start()
t3.start()

t1.join()
t3.join()

t2.start()
t2.join()


In [9]:
te = "{} minutos e {} segundos".format(int((time.time() - ts) // 60), int((time.time() - ts) % 60))
print(te)

3 minutos e 49 segundos
